# Grouping and More Joins

### 0. Setup

In [1]:
from IPython.display import Image
from IPython.core.display import HTML
%load_ext sql
%sql postgresql://appdev@data:5432/appdev

'Connected: appdev@appdev'


### 1. Using a single join and a grouping, write a query that allows you to find the forename, surname, driverid and the total number of wins for each driver in the f1db schema.

In [16]:
%%sql 
SELECT drivers.driverid as driverid,
  drivers.forename as forename,
  drivers.surname as surname,
  count(results.position) as wins
FROM drivers
JOIN results USING (driverid)
WHERE results.position = 1 
GROUP BY (drivers.driverid) 
ORDER BY surname
LIMIT 10;

10 rows affected.


driverid,forename,surname,wins
105,Michele,Alboreto,5
55,Jean,Alesi,1
4,Fernando,Alonso,32
207,Mario,Andretti,12
163,René,Arnoux,7
647,Alberto,Ascari,13
394,Giancarlo,Baghetti,1
385,Lorenzo,Bandini,1
22,Rubens,Barrichello,11
306,Jean-Pierre,Beltoise,1


The SQL statement creates this list of drivers containing the specified information. I chose to limit it to only 10 entires, but removing the limiter will give all 107 results.

### 2. Write a query that joins the tables results, constructors and drivers to show the amount of times a driver has driven a car from a constructor. In other words how many times one driver (for instance Schumacher) has driven a constructor (for instance Mercedes). Your table should have three fields: drivers.driverref, constructors.name and count(*) (showing the number of times the driver has driven the constructor).

In [30]:
%%sql
SELECT 
    driverref as Driver, 
    constructors.name as Constructor,
    count(*) as Time
FROM drivers 
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
GROUP BY (drivers.driverref, constructors.name)
ORDER BY time DESC
Limit 10;

10 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136
hakkinen,McLaren,133
laffite,Ligier,132
webber,Red Bull,129
raikkonen,Ferrari,126
vettel,Red Bull,113


### 3. Now we have found the most driven vehicles even for those who didn't finish the race. Extend the query from 2 by removing all tuples from the result where the status is not "Finished".

In [32]:
%%sql
SELECT 
    driverref as Driver, 
    constructors.name as Constructor,
    count(*) as Time
FROM drivers 
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY (drivers.driverref, constructors.name)
ORDER BY time DESC
Limit 10;

10 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97
raikkonen,Ferrari,92
button,McLaren,87
alonso,Ferrari,87
coulthard,McLaren,85
hamilton,Mercedes,85


### 4. Create a third (and new) query that finds the amount of milliseconds spent in pitstop (see the pitstops relation) by each unique combinations of driverid and raceid.

In [38]:
%%sql
CREATE VIEW pitstop_time as 
SELECT 
    driverid as drivers_id,
    races.raceid as race_id,
    driverref as driver,
    count(pitstops.driverid) as pit_stops,
    sum(pitstops.milliseconds) as milliseconds
FROM drivers
JOIN pitstops USING (driverid)
JOIN races USING (raceid)
GROUP BY (drivers.driverid, races.raceid)
ORDER BY milliseconds


Done.


[]

In [39]:
%sql SELECT * FROM pitstop_time limit 15

15 rows affected.


drivers_id,race_id,driver,pit_stops,milliseconds
154,909,grosjean,1,18928
17,877,webber,1,19066
4,877,alonso,1,19368
18,877,button,1,19455
155,877,kobayashi,1,19529
13,877,massa,1,19585
18,871,button,1,19668
822,955,bottas,1,19732
10,877,glock,1,19884
3,955,rosberg,1,20058


### 5. Time to put everything together. Use the query for 4 as a join subquery in query 3 (see slides on "a query within a query") to find the total amount of pitstop time, for each result in query 3

In [42]:
%%sql
DROP VIEW IF EXISTS finished_vehicles;
CREATE VIEW finished_vehicles AS
SELECT driverref as driver,
       constructors.name as constructor,
       count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY (drivers.driverref, constructors.name)
ORDER BY time DESC;

Done.
Done.


[]

In [48]:
%%sql
SELECT fv.driver,
       fv.constructor,
       sum(pt.milliseconds) as time
FROM finished_vehicles as fv
INNER JOIN (SELECT
              drivers_id,
              race_id,
              driver,
              pitstop_time.milliseconds,
              results.constructorid,
              constructors.name
          FROM pitstop_time
          JOIN results ON results.driverid = pitstop_time.drivers_id
          AND results.raceid = pitstop_time.race_id
          JOIN constructors ON constructors.constructorid = results.constructorid) as pt
ON pt.driver = fv.driver AND pt.name = fv.constructor
GROUP BY (fv.driver, fv.constructor)
ORDER BY time DESC
LIMIT 10;

10 rows affected.


driver,constructor,time
hamilton,Mercedes,12428578
rosberg,Mercedes,12390412
perez,Force India,11876732
massa,Williams,11874456
ricciardo,Red Bull,11816929
button,McLaren,11800713
bottas,Williams,10665109
hulkenberg,Force India,10198930
vettel,Ferrari,9248461
nasr,Sauber,8513533


### 6. Find the driver with the least pitstop time

In [58]:
%%sql
SELECT 
    driver, 
    AVG(milliseconds) as average 
FROM
    pitstop_time
GROUP BY
    (driver)
ORDER BY 
    average ASC 
LIMIT 1;

1 rows affected.


driver,average
gasly,25044.000000000000
